# AWS Starter
* StelllarAlgo Data Science
* Peter Morrison
* May 30, 2022

This notebook does the following:
1. Reads records from `datascience.ds.dummytable` in qa-app in Redshift and displays in Pandas DF
2. Appends a record to the table, then displays only that record from the table
3. Updates an existing record, then displays only that record from the table
4. Deletes the new record that I created in step 2

In [30]:
import awswrangler as wr
import boto3
import pandas as pd

from IPython.display import display

In [25]:
! aws sso login --profile Stellaralgo-DataScienceAdmin

Attempting to automatically open the SSO authorization page in your default browser.
If the browser does not open or you wish to use a different device to authorize this request, open the following URL:

https://device.sso.us-east-1.amazonaws.com/

Then enter the code:

MBVM-RJZT
Successfully logged into Start URL: https://stellaralgo.awsapps.com/start#/


In [28]:
DUMMY_TABLE = 'dummytable'

session = boto3.setup_default_session(profile_name='Stellaralgo-DataScienceAdmin')
client = boto3.client('redshift')

cluster = 'qa-app'
dbname = 'datascience'
schema = 'ds'
    

conn = wr.data_api.redshift.connect(
    cluster_id = cluster,
    database = dbname,
    db_user = "admin"
)


In [51]:
def execute_sql(statement: str) -> pd.DataFrame:
    """Executes the passed SQL statement on Redshift.

    Returns:
        pd.DataFrame: Redshift result as a dataframe.
    """
    return wr.data_api.redshift.read_sql_query(
                sql = statement, 
                con = conn
        )

## Reading Data

Here we read all data from the table in redshift, satisfying requirement 1

In [52]:
read_all_statement = f'SELECT * FROM {dbname}.{schema}.{DUMMY_TABLE}'

# We call the execute_sql func to reduce code duplication when reading/writing from/to Redshift
df = execute_sql(read_all_statement)
display(df)

,playerid,dob,gamesplayed,injured,position,name,numassists,numgoals,pointpercentage
0,11,1988-08-04,20,false,RW,Dale,24,21,2.1
1,12,1985-06-05,20,false,C,Skip,15,36,2.5
2,13,1985-03-15,15,true,LW,Sanders,20,30,1.9
3,14,1983-02-20,20,false,LD,Patty,38,12,1.5
4,15,1987-08-04,18,false,RD,Reynolds,16,6,0.8


## Writing Data
Here we write a new record to the table and read that record

In [39]:
insert_record_statement = f"""INSERT INTO {dbname}.{schema}.{DUMMY_TABLE}(
    dob, gamesplayed, injured, position, name, numassists, numgoals, pointpercentage
) VALUES (
    '1996-03-18', 0, false, 'IDK', 'Peter', 20, 2, 0.6
)"""

execute_sql(insert_record_statement)

read_new_record_statement = f'SELECT * FROM {dbname}.{schema}.{DUMMY_TABLE} WHERE playerid = 16'
df = execute_sql(read_new_record_statement)
display(df)

,playerid,dob,gamesplayed,injured,position,name,numassists,numgoals,pointpercentage
0,16,1996-03-18,0,false,IDK,Peter,20,2,0.6


## Updating Data
Here we update a record (we will update the newly made one, id=16) to change the name to Helloworld

In [40]:
update_record_statement = f"""UPDATE {dbname}.{schema}.{DUMMY_TABLE}
    SET name = 'Helloworld'
    WHERE playerid = 16
"""

execute_sql(update_record_statement)

read_new_record_statement = f'SELECT * FROM {dbname}.{schema}.{DUMMY_TABLE} WHERE playerid = 16'
df = execute_sql(read_new_record_statement)
display(df)

,playerid,dob,gamesplayed,injured,position,name,numassists,numgoals,pointpercentage
0,16,1996-03-18,0,false,IDK,Helloworld,20,2,0.6


## Deleting Data
Here we delete the record we made (id=16) and display the full table.

In [41]:
delete_record_statement = f"""DELETE FROM {dbname}.{schema}.{DUMMY_TABLE}
    WHERE playerid = 16
"""

execute_sql(delete_record_statement)

read_all_statement = f'SELECT * FROM {dbname}.{schema}.{DUMMY_TABLE}'
df = execute_sql(read_all_statement)
display(df)

,playerid,dob,gamesplayed,injured,position,name,numassists,numgoals,pointpercentage
0,11,1988-08-04,20,false,RW,Dale,24,21,2.1
1,12,1985-06-05,20,false,C,Skip,15,36,2.5
2,13,1985-03-15,15,true,LW,Sanders,20,30,1.9
3,14,1983-02-20,20,false,LD,Patty,38,12,1.5
4,15,1987-08-04,18,false,RD,Reynolds,16,6,0.8
